# Exploring_Join_dataset

## Content
- [VIVIENDA](#VIVIENDA)
- [HOGAR](#HOGAR)
- [PERSONAS](#PERSONAS)
- [FALLECIDOS](#FALLECIDOS)
- [GEOREFERENCIACION](#GEOREFERENCIACION)
- [Divipola](#Divipola)

### Aggregations by city


## Summary about  each dataset


In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import os
import configparser
import boto3
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

%load_ext autoreload
%autoreload 2

In [2]:
os.chdir(os.path.join(os.getcwd(), ".."))

In [3]:
from src.config import metadata
from src.utils.s3_utils import (get_censo_paths, download_files_from_s3, read_multiple_csv,
                                mk_partitioned_dir, identified_partitioned_dir)
from src.utils.general_utils import create_dir

In [4]:
config = configparser.ConfigParser()
config.read('aws.cfg')
aws = config["AWS"]
bucket = "censo-covid"
s3 = boto3.resource('s3',
                    region_name="us-west-2",
                    aws_access_key_id=aws['AWS_ACCESS_KEY_ID'],
                    aws_secret_access_key=aws['AWS_SECRET_ACCESS_KEY'])
s3_client = boto3.client('s3',
                         aws_access_key_id=aws['AWS_ACCESS_KEY_ID'],
                         aws_secret_access_key=aws['AWS_SECRET_ACCESS_KEY']
                        )
data_path = os.path.join(os.getcwd(), "data")
censo_data_path = os.path.join(data_path, "censo")
covid_file_path = os.path.join(os.getcwd(), "data", "covid.csv")
divipola_file_path = os.path.join(os.getcwd(), "data", "divipola.csv")
fallecidos_complete_path = os.path.join(data_path, "fallecidos")
aggregates_path = os.path.join(data_path,'aggregates')
joins_path = os.path.join(data_path,'joins')

aggregates_personas_path = os.path.join(aggregates_path, 'personas')
joins_personas_path = os.path.join(joins_path, 'personas')

aggregates_fallecidos_path = os.path.join(aggregates_path,'fallecidos')
joins_fallecidos_path = os.path.join(data_path,'fallecidos')
s3Bucket = s3.Bucket(bucket)

In [5]:
create_dir(aggregates_path)
create_dir(joins_path)
create_dir(aggregates_personas_path)
create_dir(joins_personas_path)
create_dir(aggregates_fallecidos_path)
create_dir(joins_fallecidos_path)

/Users/camilovelasquez/Desktop/Documents/Camilo/Udacity/censo_covid_colombia/data/aggregates dir exists
/Users/camilovelasquez/Desktop/Documents/Camilo/Udacity/censo_covid_colombia/data/joins dir exists
/Users/camilovelasquez/Desktop/Documents/Camilo/Udacity/censo_covid_colombia/data/aggregates/personas dir exists
/Users/camilovelasquez/Desktop/Documents/Camilo/Udacity/censo_covid_colombia/data/joins/personas dir exists
/Users/camilovelasquez/Desktop/Documents/Camilo/Udacity/censo_covid_colombia/data/aggregates/fallecidos dir exists
/Users/camilovelasquez/Desktop/Documents/Camilo/Udacity/censo_covid_colombia/data/fallecidos dir exists


In [6]:
download_files_from_s3(aggregates_personas_path, s3Bucket, prefix="final-data/aggregates_personas")

Already downloaded final-data/aggregates_personas


In [7]:
list_frames = read_multiple_csv(aggregates_personas_path, to_concat=[], header=0, n_files=7)
df_agg_personas = pd.concat(list_frames)
df_agg_personas.head()

,nom_mpio,Number_of_people,Avg_Number_people_per_home,Number_of_males,Number_of_females,Number_of_people_with_electricity,Number_of_people_without_electricity,Number_of_people_with_water_access,Number_of_people_without_water_access,Number_of_people_with_internet_access,Number_of_people_without_internet_access,Number_of_literate_people,Number_of_non_literate_people,Really_Good_health_service,Good_health_service,Bad_health_service,Really_Bad_health_service,Number_of_sons_out_of_country,VA1_ESTRATO_0,VA1_ESTRATO_1,VA1_ESTRATO_2,VA1_ESTRATO_3,VA1_ESTRATO_4,VA1_ESTRATO_5,VA1_ESTRATO_6,VA1_ESTRATO_9,P_NIVEL_ANOSR_1,P_NIVEL_ANOSR_2,P_NIVEL_ANOSR_3,P_NIVEL_ANOSR_4,P_NIVEL_ANOSR_5,P_NIVEL_ANOSR_6,P_NIVEL_ANOSR_7,P_NIVEL_ANOSR_8,P_NIVEL_ANOSR_9,P_NIVEL_ANOSR_10,P_NIVEL_ANOSR_99,P_EDADR_1,P_EDADR_2,P_EDADR_3,P_EDADR_4,P_EDADR_5,P_EDADR_6,P_EDADR_7,P_EDADR_8,P_EDADR_9,P_EDADR_10,P_EDADR_11,P_EDADR_12,P_EDADR_13,P_EDADR_14,P_EDADR_15,P_EDADR_16,P_EDADR_17,P_EDADR_18,P_EDADR_19,P_EDADR_20,P_EDADR_21,dpto
0,BOYACÁ,4472,3.946780,2307,2165,4357,115,3784,688,263,4191,3652,403,18,312,119,3.0,170.0,27,1387,2886,37,7,0,0,13,66,2229,629,513,60,3,108,82,27,326,168,261,327,418,437,283,269,262,306,251,274,242,244,220,203,163,140,94,64,13,1,0,BOYACÁ
1,BUSBANZÁ,903,3.667411,446,457,861,35,825,71,44,843,776,53,6,30,10,3.0,17.0,2,641,207,11,0,0,0,0,21,359,130,174,25,3,41,23,11,40,7,69,73,76,79,66,59,67,65,46,54,41,35,29,38,36,28,24,12,4,2,0,BOYACÁ
2,LA VICTORIA,956,3.403766,504,452,877,79,631,325,49,903,790,118,5,17,3,NaN,8.0,3,674,183,16,0,1,0,0,26,458,120,110,13,2,35,25,19,100,8,40,75,69,106,55,49,53,60,80,67,60,60,46,36,33,35,27,4,1,0,0,BOYACÁ
3,MONIQUIRÁ,20848,3.713988,10312,10536,20412,255,16348,4319,4455,16092,17760,1643,73,1059,328,89.0,625.0,19,5703,10895,3518,210,14,0,53,335,7281,3173,3826,497,67,1000,1423,508,1268,231,1239,1452,1772,1724,1407,1324,1231,1357,1310,1315,1304,1241,1173,960,804,572,353,210,77,21,2,BOYACÁ
4,NUEVO COLÓN,5025,3.945687,2587,2438,4777,231,3969,1039,216,4739,4356,343,18,273,194,110.0,380.0,3,620,4125,15,2,0,0,12,116,2371,760,707,104,7,183,170,33,240,26,308,344,479,450,325,255,300,337,334,340,309,262,286,234,184,128,87,43,17,0,3,BOYACÁ


## Reading covid data

In [8]:
df = pd.read_csv(covid_file_path)
df.head()

/Users/camilovelasquez/anaconda3/envs/censo/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3072: DtypeWarning: Columns (9,12,14,17,18,19) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,fecha_de_notificaci_n,c_digo_divipola,ciudad_de_ubicaci_n,departamento,atenci_n,edad,sexo,tipo,estado,pa_s_de_procedencia,fis,fecha_diagnostico,fecha_recuperado,fecha_reporte_web,tipo_recuperaci_n,codigo_departamento,codigo_pais,pertenencia_etnica,nombre_grupo_etnico,fecha_de_muerte,Asintomatico,divipola_dpto,divipola_mpio,edad_q,muerto,edad_muerto
0,2020-03-02,11001,Bogotá D.C.,Bogotá D.C.,Recuperado,19,F,Importado,Leve,ITALIA,2020-02-27,2020-03-06,2020-03-13,2020-03-06,PCR,11,380.0,Otro,NaN,NaN,0,11,1,1,False,0
1,2020-03-06,76111,Guadalajara de Buga,Valle del Cauca,Recuperado,34,M,Importado,Leve,ESPAÑA,2020-03-04,2020-03-09,2020-03-19,2020-03-09,PCR,76,724.0,Otro,NaN,NaN,0,76,111,2,False,0
2,2020-03-07,5001,Medellín,Antioquia,Recuperado,50,F,Importado,Leve,ESPAÑA,2020-02-29,2020-03-09,2020-03-15,2020-03-09,PCR,5,724.0,Otro,NaN,NaN,0,5,1,3,False,0
3,2020-03-09,5001,Medellín,Antioquia,Recuperado,55,M,Relacionado,Leve,NaN,2020-03-06,2020-03-11,2020-03-26,2020-03-11,PCR,5,NaN,Otro,NaN,NaN,0,5,1,4,False,0
4,2020-03-09,5001,Medellín,Antioquia,Recuperado,25,M,Relacionado,Leve,NaN,2020-03-08,2020-03-11,2020-03-23,2020-03-11,PCR,5,NaN,Otro,NaN,NaN,0,5,1,5,False,0


In [10]:
df.dtypes

fecha_de_notificaci_n     object
c_digo_divipola            int64
ciudad_de_ubicaci_n       object
departamento              object
atenci_n                  object
edad                       int64
sexo                      object
tipo                      object
estado                    object
pa_s_de_procedencia       object
fis                       object
fecha_diagnostico         object
fecha_recuperado          object
fecha_reporte_web         object
tipo_recuperaci_n         object
codigo_departamento        int64
codigo_pais              float64
pertenencia_etnica        object
nombre_grupo_etnico       object
fecha_de_muerte           object
Asintomatico               int64
divipola_dpto              int64
divipola_mpio              int64
edad_q                     int64
muerto                      bool
edad_muerto                int64
dtype: object

In [11]:
df_divipola = pd.read_csv(divipola_file_path)
df_divipola.head()

,cod_depto,cod_mpio,dpto,nom_mpio,tipo_municipio
0,5,1,ANTIOQUIA,MEDELLÍN,Municipio
1,5,2,ANTIOQUIA,ABEJORRAL,Municipio
2,5,4,ANTIOQUIA,ABRIAQUÍ,Municipio
3,5,21,ANTIOQUIA,ALEJANDRÍA,Municipio
4,5,30,ANTIOQUIA,AMAGÁ,Municipio


In [12]:
df = df.merge(df_divipola, 
              left_on=["divipola_dpto", "divipola_mpio"], 
              right_on=["cod_depto", "cod_mpio"], how="left")

In [13]:
df_atencion = df.groupby(["dpto", "nom_mpio", "atenci_n"])["atenci_n"].count().unstack(2).fillna(0)
assert abs(df_atencion.sum().sum() - len(df)) <= 0.1*len(df)

In [14]:
df_edad_q = df.groupby(["dpto", "nom_mpio", "edad_q"])["edad_q"].count().unstack(2).fillna(0)
df_edad_q.columns = ["edadq_" + str(col) for col in df_edad_q.columns]
assert abs(df_edad_q.sum().sum() - len(df)) <= 0.1*len(df)

In [15]:
df_sexo = df.groupby(["dpto", "nom_mpio", "sexo"])["sexo"].count().unstack(2).fillna(0)
assert abs(df_sexo.sum().sum() - len(df)) <= 0.1*len(df)

In [16]:
df_estado = df.groupby(["dpto", "nom_mpio", "estado"])["estado"].count().unstack(2).fillna(0)
assert abs(df_estado.sum().sum() - len(df)) <= 0.1*len(df)

In [17]:
df_muertos = df.groupby(["dpto", "nom_mpio", "muerto"])["muerto"].count().unstack(2).fillna(0)
df_muertos.columns = ["muerto_" + str(col) for col in df_muertos.columns]
assert abs(df_muertos.sum().sum() - len(df)) <= 0.1*len(df)

In [18]:
df_edad_muerto = df.groupby(["dpto", "nom_mpio"])["edad_muerto"].sum()\
                .to_frame().rename(columns={"edad_muerto": "sum_edad_muerto"})

In [19]:
result = pd.concat([df_atencion, df_edad_q, df_sexo, df_estado, df_muertos, df_edad_muerto], axis=1)
result.head()

Casa  Fallecido  Hospital  Hospital UCI  Recuperado  edadq_1  edadq_2  edadq_3  edadq_4  edadq_5  edadq_6  edadq_7  edadq_8  edadq_9  edadq_10  edadq_11  edadq_12  edadq_13  edadq_14  edadq_15  edadq_16  edadq_17  edadq_18  edadq_19  edadq_20  edadq_21      F       M  Asintomático  Fallecido  Grave    Leve  Moderado  muerto_False  muerto_True  sum_edad_muerto
dpto      nom_mpio                                                                                                                                                                                                                                                                                                                                                                                   
AMAZONAS  LA CHORRERA        1.0        0.0       2.0           1.0         0.0      0.0      0.0      0.0      0.0      0.0      0.0      0.0      1.0      1.0       0.0       0.0       0.0       0.0       1.0       0.0       1.0       0.0       0.0       0.0       0.0       0.0    1.0     3.0           0.0        0.0    1.0     1.0       2.0           4.0          0.0                0
          LETICIA           75.0       98.0       6.0           6.0      2125.0    238.0    220.0    141.0    121.0    233.0     12.0    294.0    230.0    161.0      65.0      77.0     150.0      86.0      43.0      68.0      34.0      33.0      97.0       7.0       1.0       0.0  973.0  1338.0         308.0       98.0    6.0  1889.0       9.0        2212.0         99.0             6822
          PUERTO NARIÑO      7.0        3.0       2.0           0.0       105.0     15.0     10.0      7.0      5.0     13.0      0.0     20.0     14.0      8.0       0.0       2.0       7.0       1.0       1.0       2.0       2.0       1.0       9.0       0.0       0.0       0.0   47.0    70.0          16.0        3.0    0.0    96.0       2.0         114.0          3.0              165
          PUERTO SANTANDER   NaN        NaN       NaN           NaN         NaN      0.0      0.0      0.0      0.0      0.0      0.0      0.0      0.0      0.0       0.0       0.0       0.0       0.0       0.0       0.0       0.0       0.0       1.0       0.0       0.0       0.0    0.0     1.0           NaN        NaN    NaN     NaN       NaN           0.0          1.0               10
ANTIOQUIA AMAGÁ              5.0        1.0       0.0           0.0         4.0      0.0      1.0      1.0      1.0      1.0      0.0      2.0      2.0      0.0       0.0       0.0       1.0       0.0       0.0       0.0       0.0       1.0       0.0       0.0       0.0       0.0    7.0     3.0           0.0        1.0    0.0     9.0       0.0           9.0          1.0               55

## Join

In [124]:
joined_personas = df_agg_personas.merge(result, left_on=["dpto", "nom_mpio"], right_index=True, how="inner")
joined_personas.head()

,nom_mpio,Number_of_people,Avg_Number_people_per_home,Number_of_males,Number_of_females,Number_of_people_with_electricity,Number_of_people_without_electricity,Number_of_people_with_water_access,Number_of_people_without_water_access,Number_of_people_with_internet_access,Number_of_people_without_internet_access,Number_of_literate_people,Number_of_non_literate_people,Really_Good_health_service,Good_health_service,Bad_health_service,Really_Bad_health_service,Number_of_sons_out_of_country,VA1_ESTRATO_0,VA1_ESTRATO_1,VA1_ESTRATO_2,VA1_ESTRATO_3,VA1_ESTRATO_4,VA1_ESTRATO_5,VA1_ESTRATO_6,VA1_ESTRATO_9,P_NIVEL_ANOSR_1,P_NIVEL_ANOSR_2,P_NIVEL_ANOSR_3,P_NIVEL_ANOSR_4,P_NIVEL_ANOSR_5,P_NIVEL_ANOSR_6,P_NIVEL_ANOSR_7,P_NIVEL_ANOSR_8,P_NIVEL_ANOSR_9,P_NIVEL_ANOSR_10,P_NIVEL_ANOSR_99,P_EDADR_1,P_EDADR_2,P_EDADR_3,P_EDADR_4,P_EDADR_5,P_EDADR_6,P_EDADR_7,P_EDADR_8,P_EDADR_9,P_EDADR_10,P_EDADR_11,P_EDADR_12,P_EDADR_13,P_EDADR_14,P_EDADR_15,P_EDADR_16,P_EDADR_17,P_EDADR_18,P_EDADR_19,P_EDADR_20,P_EDADR_21,dpto,Casa,Fallecido,Hospital,Hospital UCI,Recuperado,edadq_1,edadq_2,edadq_3,edadq_4,edadq_5,edadq_6,edadq_7,edadq_8,edadq_9,edadq_10,edadq_11,edadq_12,edadq_13,edadq_14,edadq_15,edadq_16,edadq_17,edadq_18,edadq_19,edadq_20,edadq_21,F,M,Asintomático,Fallecido,Grave,Leve,Moderado,muerto_False,muerto_True,sum_edad_muerto
3,MONIQUIRÁ,20848,3.713988,10312,10536,20412,255,16348,4319,4455,16092,17760,1643,73,1059,328,89.0,625.0,19,5703,10895,3518,210,14,0,53,335,7281,3173,3826,497,67,1000,1423,508,1268,231,1239,1452,1772,1724,1407,1324,1231,1357,1310,1315,1304,1241,1173,960,804,572,353,210,77,21,2,BOYACÁ,4.0,0.0,0.0,0.0,5.0,0.0,0.0,0.0,2.0,0.0,2.0,0.0,2.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,6.0,3.0,1.0,0.0,0.0,7.0,1.0,9.0,0.0,0
4,NUEVO COLÓN,5025,3.945687,2587,2438,4777,231,3969,1039,216,4739,4356,343,18,273,194,110.0,380.0,3,620,4125,15,2,0,0,12,116,2371,760,707,104,7,183,170,33,240,26,308,344,479,450,325,255,300,337,334,340,309,262,286,234,184,128,87,43,17,0,3,BOYACÁ,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0
5,PAIPA,31141,3.845483,15006,16135,30838,259,29492,1605,8162,22779,27760,1070,172,2115,584,77.0,873.0,16,3852,23816,2814,255,30,0,55,628,9446,4792,5698,1054,68,2241,3290,844,695,425,1960,2211,2693,2823,2454,2094,2103,2222,2040,1980,1879,1760,1475,1174,833,655,430,225,95,28,7,BOYACÁ,0.0,1.0,1.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,2.0,0.0,0.0,2.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,4.0,2.0,0.0,1.0,0.0,2.0,1.0,5.0,1.0,80
7,TUNJA,167991,3.843536,79244,88747,166255,595,164712,2138,97699,67122,149802,3958,759,9134,3916,943.0,4989.0,102,23478,53452,60856,19205,8755,92,315,3270,32200,21137,30607,4016,803,14096,29556,14337,3156,4044,10769,11965,13333,15263,16732,14132,13262,13076,11214,10257,9522,8649,6833,4866,3091,2207,1570,830,304,78,38,BOYACÁ,43.0,4.0,12.0,0.0,91.0,5.0,18.0,7.0,7.0,16.0,1.0,19.0,13.0,15.0,3.0,6.0,13.0,8.0,1.0,6.0,2.0,5.0,3.0,3.0,0.0,0.0,71.0,80.0,44.0,4.0,0.0,91.0,12.0,146.0,5.0,286
8,VENTAQUEMADA,13984,3.883784,6937,7047,13838,136,13072,902,663,13226,12078,873,47,786,551,100.0,104.0,2,3387,10352,82,0,1,4,10,358,6242,2252,2483,130,9,379,482,87,514,63,985,1130,1320,1321,1170,859,993,959,932,881,733,632,534,488,379,298,193,95,37,8,37,BOYACÁ,1.0,0.0,0.0,0.0,6.0,1.0,0.0,0.0,1.0,3.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,5.0,2.0,0.0,0.0,5.0,0.0,7.0,0.0,0


In [125]:
joined_personas.dtypes

nom_mpio                                     object
Number_of_people                              int64
Avg_Number_people_per_home                  float64
Number_of_males                               int64
Number_of_females                             int64
Number_of_people_with_electricity             int64
Number_of_people_without_electricity          int64
Number_of_people_with_water_access            int64
Number_of_people_without_water_access         int64
Number_of_people_with_internet_access         int64
Number_of_people_without_internet_access      int64
Number_of_literate_people                     int64
Number_of_non_literate_people                 int64
Really_Good_health_service                    int64
Good_health_service                           int64
Bad_health_service                            int64
Really_Bad_health_service                   float64
Number_of_sons_out_of_country               float64
VA1_ESTRATO_0                                 int64
VA1_ESTRATO_